## Data Modeling

### Clustering

My nephew wants me to program a video game for him called 'Monster Family', and even provided a description of its characters:

<img src='https://courses.edx.org/asset-v1:Microsoft+DAT210x+4T2016+type@asset+block@Haadi.jpg'
alt='' style='height:300px;'/>

Although he's young, you can tell he's particular about details. To make the game to his likings, I needed to know which monsters were real family members, and which monsters weren't. If he had added just one more column that held that detail, FamilyA, FamilyB, etc., I would have been set:

<img src='pic/data-modeling-clustering-1.png'
alt='' style='height:200px;'/>

This monster 'dataset' is similar to real-world data in that it comes loaded with observational features, but isn't labeled. The one question I want a direct answer to isn't included as a feature. If there were way to automatically group similar samples based solely on their features, we then could use that knowledge to guide us towards actionable intelligence. That way exist, and its called unsupervised clustering.

#### Similarity

Since the goal of clustering is the grouping of similar records, you have to first define what similarity means. How would you go define monster similarity?

<img src='pic/data-modeling-clustering-2.png'
alt='' style='height:200px;'/>

One way you could group them is by attack power. Perhaps the monsters who deal the most damage to the player belong to a family, the weaker monsters belong to a family, and then the rest grouped as a family too, as shown above.

<img src='pic/data-modeling-clustering-3.png'
alt='' style='height:200px;'/>

Another grouping would be by weakness. It seems a lot of monsters that share a water-based weakness. All of these might belong to the same family. The remaining monsters might each belong to a separate family, or might actually be members of a sporadic family. There are many other ways you could group them as well, such as by size, by name (e.g. monster vs snake), etc. 

Without a generalizable way to group the samples, deterministic computers can't cluster your data. What's needed is a systematic means of measuring the **overall** similarity between your samples. Let's discuss how that's accomplished in the next section.

#### How Does k-Means Work?

Clustering groups samples that are similar within the same cluster. The more similar the samples belonging to a cluster group are (and conversely, the more dissimilar samples in separate groups), the better the clustering algorithm has performed. Since clustering is an unsupervised algorithm, this similarity metric must be measured automatically and based solely on your data.

The implementation details and definition of *similarity* are what differentiate the many clustering algorithms. The **K-Means** way of doing this, is to iteratively separate your samples into a user-specified number of "K" cluster groups of roughly equal variance. Cluster groups are defined by their geometric cluster center, single point referred to as its centroid. Separately, *centroid* and *cluster* are sometimes used interchangeably; but if used together, a cluster is a set of similar samples, and a centroid is just the mean feature-position of all samples assigned to the cluster.

The centroids are not records in your dataset, however they do 'exist' within your datasets feature-space. This is important because it allows for a meaningful distance measure to be calculated between the centroids and your samples. Every sample in your dataset is assigned to the centroid nearest to it, so if you have a sample that is 10 units away from CusterA's centroid, and 100 units away from ClusterB's, the sample is assigned to ClusterA.

In the case of continuous features, calculating the distance is straightforward. But when you have categorical features, such as 'Cookies n Cream' vs 'Mango' ice cream favors, you'll have to creatively come up with other methods. SciKit-Learn's K-Means implementation only natively supports numeric features types, so we'll leave the discussion on how to do clustering with categorical features to the Dive Deeper section.

#### The K-Means Algorithm

K-Means starts by placing a user-specified number of "K" cluster centers in your feature space. There are many techniques for choosing the first centroid placement, and your results will vary depending on the one you select! The simplest being just use the position of some random samples as the centroids' starting spots.

Each cluster then takes ownership of the samples nearest to its centroid, and every sample can only be assigned as single cluster. 'Nearest' is a value that has to be evaluated and in SciKit-Learn, it is defined as the multivariate, n-dimensional Euclidean distance between the sample and the centroid. After this, the centroid location is updated to be the mean value of all samples assigned to it. This mean value is calculated by feature, so the centroid position ends up being a n-length vector within your feature space.

The assignment and update steps repeat until there are no more changes in either, at which point the algorithm has converged. K-Means always converges, and it is very fast at doing so. But it does not always converge at the global minima...

The technical explanation for what K-Means does is minimizing the within-cluster inertia, or **sum of squared errors** between each sample and its respective centroid. As mentioned, the initial centroid assignment affects the results. Two runs of K-means might produce different outcomes, but the quality of their cluster assignments are ranked by looking at which run has the smallest overall inertia.

#### When Should I Use K-Means?

Clustering is a natural action we do even as children, by arranging similar shaped blocks and colors. K-Means clustering is best suited when you have a good idea of the number of distinct clusters your unlabeled dataset should be segmented into. Generally, the output of K-Means is used in two ways. To separate your unlabeled data into K groups, which is the clear use case, or to find and use the resulting centroids.

##### Separate Your Data

Astronomers use clustering to group different star types, classes of planets, and galaxies. Biologists use it to group every living thing by species, genus, and kingdom. In business, clustering is used to segment likely and unlikely prospects, for location assignment, factor endowment, and the assignment and deployment of remote services.

##### Centroid Usage

Besides divvying up samples, clustering can also provide a layer of abstraction, by directing attention to the cluster and its attributes and not each samples. In the climate change case study from the previous module, you saw how climate divisions were used as a cluster abstraction over individual ground stations for various mentioned reasons. Another example of centroid usage would be a company looking for ideal locations to open a limited number of branches, based on the location of their customers.

You can use the centroid to 'compress' your data. By referring to the centroid rather than the data sample, the number of unique values is reduced, which optimizes the execution speed of other algorithms. Isomap, for instance, uses a nearest neighbors algorithm to calculate the distance from the record you want to transform to every sample in the training dataset. By using the record-to-cluster distance approximation in replacement of the individual record-to-sample distances, since there are far fewer clusters than records, you can achieve unprecedented orders of optimization.

#### SciKit-Learn and K-Means

It's very simple to get up and running with K-Means in SKLearn. Given a dataframe df, you can compute its labels and centroids as follows:

```python
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(df)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001, verbose=0)

labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_
```

The most important factor for you to focus on being n_clusters, the "**K**" number of clusters you want K-Means to place for you. Also experiment with different initialization methods, including rolling your own and in the positions as an NDArray shaped as `[n_clusters, n_features]`. We've include more details for you on that in the dive deeper section.

The most important factor for you to focus on being n_clusters, the "**K**" number of clusters you want K-Means to place for you. Also experiment with different initialization methods, including rolling your own and in the positions as an NDArray shaped as `[n_clusters, n_features]`. We've include more details for you on that in the dive deeper section.

#### K-Means Gotchas!

t's easy to understand the K-Means algorithm, and extremely fast to execute. So fast that it's often ran several times over as you saw earlier. Since each successive run of isn't dependent on the results of earlier runs, the execution process lends itself to parallelization, each centroid seeding trial being ran independently. If the clustering job at hand is still taking too long, SciKit-Learn's [MiniBatchKMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) further optimizes the process for you.

Considering how basic of an algorithm it is, K-Means performs pretty well, and its implementation is the basis for a few more advanced clustering algorithms, such as learning vector quantization and Gaussian mixture. Having a solid understanding of K-Means will help you understand those better when you study them.

K-Means is only really suitable when you have a good estimate of the number clusters that exist in your unlabeled data. There are many estimation techniques for approximating the correct number of clusters, but you'll have to get that number before running K-Means. Even if you do have the right number of clusters selected, the result produced by K-Means can vary depending on the initial centroid placement. So if you need the same results produced each time, your centroid seeding technique also needs to be able to reliably produce the same placement given the same data. Due to the centroid seed placement having so much of an effect on your clustering outcome, you have to be careful since it is possible to have centroids with only a single sample assigned to them, or even no samples assigned to them in the worst case scenario.

Two other key characteristics of K-Means are that it assumes your samples are length normalized, and as such, is sensitive to feature scaling. It also assumes that the cluster sizes are roughly spherical and similar; this way, the nearest centroid is always the correct assignment.

#### Knowledge Checks

##### Review Question 1

*Only one of following statements is true. Which one is it?*

+ **Its possible for samples from two different clusters to be more similar to one another than their intra-cluster neighbors, if the the two clusters are large and located near one another correct**
+ Real world data typically comes labeled
+ Unsupervised clustering aims to group your samples based on their labels
+ Centroids are records that live in your dataset and share the same feature space so that a meaningful distance can be calculated between them and your samples

##### Review Question 2

Once again, only a single one of the following statements is correct. Do you know which one it is?*

+ It's possible for a sample to be assigned to two clusters; but only if its equidistant from either cluster.
+ The K-Means algorithm scans your dataset to detect clusters using an iterative assignment / update cycle. The algorithm returns the number of clusters found, as well as their centroid position. incorrect
+ As a clustering algorithm, K-Means is really only useful for grouping your samples
+ **K-Means assumes your features are either length normalized, or that their length encodes a specific meaning.**

*Answer*

**Incorrect**: 

Wrong! You have to specify how many clusters exist in your data. Given that number, K-Means will attempt to find *that* many clusters in your data. But the responsibility of specifying the number of clusters is yours, not the algorithm's.

**Explanation**

A sample can only have a single cluster assignment. Also, you are the one responsible for specifying the number of clusters. K-Means won't tell you the number of clusters in your data. Besides assigning a cluster to your samples, there are many uses for the centroid locations. Review th reading please. Since K-Means cluster assignment depends on an Euclidean length metric, your features have to either be length normalized, or have appropriate units for the algorithm to perform properly.

#### Assignment 1

##### Lab Assignment 1

Many U.S. cities, the U.S. federal government, and even other cities and governments abroad have started subscribing to an Open Data policy, because some data should be transparent and available to everyone to use and republish freely, without restrictions from copyright, patents, or other mechanisms of control. After reading their [terms of use](http://www.cityofchicago.org/city/en/narr/foia/data_disclaimer.html), in this lab you'll be exploring the City of Chicago's Crime data set, which is part of their Open Data initiative.

1. Start by navigating over to the [City of Chicago's Crimes dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2) exploration page. It lists crimes from 2001 to the present, but you'll only be targeting Gambling. The city's website itself has hundreds of other datasets you can browse and do machine learning on.
2. Open up the /Module5/**assignment1.py** starter code, and follow the directions to acquire the dataset and properly set it up.
3. Fill out the **doKMeans** method to find and plot **seven clusters** and print out their centroids. These could be places a police officer investigates to check for on-going illegal activities.
4. Re-run your assignment a few times over, looking at your printed and plotted results. Then answer the following questions.

*Note: If Pandas complains about your data, you can use dropna() on any row that has nans in it.*

##### Lab Questions

2 points possible (graded)

You'll notice that the cluster assignments are pretty accurate. Most of them should be spot-on, dead-center. Only one cluster might have been assigned to outliers. Given the results, answer the following questions to the best of your ability:

*Did your centroid locations change after you limited the date range to +2011?*

+ Their locations are completely different
+ **They move slightly...**
+ Not at all

*What about during successive runs of your assignment? Any centroid location changes happened there?*

+ All clusters have moved, and the cluster arrangement isn't anything like it was before
+ **All clusters have moved but only slightly, and the centroid arrangement still has the same shape for the most part**
+ The clusters did not really move at all, or if they did, it wasn't noticeable
+ The cluster centroids are identical according to the print statement output

#### Assigment 2

##### Lab Assignment 2

The spirit of data science includes exploration, traversing the unknown, and applying a deep understanding of the challenge you're facing. In an academic setting, it's hard to duplicate these tasks, but this lab will attempt to take a few steps away from the traditional, textbook, "plug the equation in" pattern, so you can get a taste of what analyzing data in the real world is all about.

After the September 11 attacks, a series of secret regulations, laws, and processes were enacted, perhaps to better protect the citizens of the United States. These processes continued through president Bush's term and were renewed and and strengthened during the Obama administration. Then, on May 24, 2006, the United States Foreign Intelligence Surveillance Court (FISC) made a fundamental shift in its approach to Section 215 of the Patriot Act, permitting the FBI to compel production of "business records" relevant to terrorism investigations, which are shared with the NSA. The court now defined as *business* records the entirety of a telephone company's call database, also known as Call Detail Records (**CDR** or *metadata*).

News of this came to public light after an ex-NSA contractor leaked the information, and a few more questions were raised when it was further discovered that not just the call records of suspected terrorists were being collected in bulk... but perhaps the entirety of Americans as a whole. After all, if you know someone who knows someone who knows someone, your private records are relevant to a terrorism investigation. The white house quickly reassured the public in [a press release](http://www.cbsnews.com/news/obama-nobody-is-listening-to-your-telephone-calls/) that "Nobody is listening to your telephone calls," since, "that's not what this program is about." The public was greatly relieved.

The questions you'll be exploring in this lab assignment using K-Means are: exactly how useful is telephone metadata? It must have some use, otherwise the government wouldn't have invested however many millions they did into it secretly collecting it from phone carriers. Also what kind of intelligence can you extract from CDR metadata besides its face value?

You will be using a sample CDR dataset generated for 10 people living in the Dallas, Texas metroplex area. Your task will be to attempt to do what many researchers [have already](http://journals.plos.org/plosone/article/asset?unique&id=info:doi/10.1371/journal.pone.0020814.s001) successfully done - partly de-anonymize the CDR data. People generally behave in predictable manners, moving from home to work with a few errands in between. With enough call data, given a few K-locations of interest, K-Means should be able to isolate rather easily the geolocations where a person spends the most of their time.

Note: to safeguard from doxing people, the CDR dataset you'll be using for this assignment was generated using the tools available in the Dive Deeper section. CDRs are at least supposed to be protected by privacy laws, and are the basis for proprietary revenue calculations. In reality, there are quite a few public CDRs out there. Much information can be discerned from them such as social networks, criminal acts, and believe it or not, even the spread of diseases as was demonstrated by [Flowminder Foundation paper on Ebola](http://www.worldpop.org.uk/ebola/Flowminder-Mobility-Data-21.08.14.pdf). 

1. Open up the starter code in /Module5/*assignment2.py* and *read* through it all. It's long, so make sure you understand everything that is being asked for you before proceeding.
2. Load up the CDR dataset from /Module5/Datasets/**CDR.csv**. Do your due diligence to make sure it's been loaded correctly and all the features and rows match up.
3. Pick the first unique user in the list to examine. Follow the steps in the assignment file to approximate where the user lives.
4. Once you have a (**Latitude**, **Longitude**) coordinate pair, drop them into Google Maps. Just do a search for the "{Lat, Lon}". So if your centroid is located at Longitude = **-96.949246** and Latitude = **32.953856**, then do a maps search for "[32.953856, -96.949246](https://www.google.com/maps/place/32%C2%B057'13.9%22N+96%C2%B056'57.3%22W/@32.953856,-96.950343,18z/data=!3m1!4b1!4m5!3m4!1s0x0:0x0!8m2!3d32.953856!4d-96.949246)".
5. Answer the questions below.

#### Assignment 3

##### Lab Assignment 3

Continuing on with the previous lab, this time you'll validate your results by comparing the user's weekday activity to their weekend activity. To get started, use the starter code in /Module5/**assignment3.py**.

Load up the same CDR dataset into a dataframe, and extract the unique "**In**" phone numbers. You don't have to save it as a Python list this time, and can keep it as an NDArray. The previous lab had you convert to a list just so you'd have the experience doing it.
Create a new slice, once again for the first unique number in the CDR. Instead of limiting it to Weekend only entries, index it so that the slice only contains Weekday entries, **Mon-Fri**, and so that it occurs any time before 5pm.
Run K-Means on the data with K=4. Plot the cellphone towers the user connected to, and then plot the cluster centers using a different marker and color.
Answer the questions below.

##### Lab Questions
3/3 points (graded)
Answer the following questions given the data you just recorded, for K=4, and CallTime is less than **5pm** (that is "*17:00:00*"), and the call's day-of-week being a weekday.

The users home location will likely be near the centroid with the second most attached samples. Does your approximated home location from this map coincide with the home approximation from the previous lab?

+ Yes, they are exactly the same
+ **Yes, they match, but there is a slight difference correct**
+ No, you can tell that they should match; however, their locations are *very* different
+ No, they are completely different

Given the indexed time range, and the times people usually receive / make calls, the cluster with the most samples is likely to be the user's work location. What is the phone number of the user who works at the US Post Office near Cockrell Hill Rd?

+ 463-847-2273
+ 206-862-7935
+ **289-436-5987**
+ 155-941-0755
+ 368-808-9071

Run your assignment with K=3. Look at the code that gets the mean CallTime value for the cluster with the least amount of samples assigned to it (the cluster we suspect corresponds to the user transiting to work). What hour is the average CallTime value of that cluster closest to?

+ 5am
+ 6am
+ 7am
+ **8am**
+ 9am
+ 10am
+ 11am
 
**CallTime Clarification**

Your calculated average calltime should be on a *per-cluster* basis. Recall, each user in your dataset has 3 clusters, and you're only interested in the cluster-per-user with the fewest # of samples--that is, the least number of `.lables_`. So you should have 10 clusters total, each with a certain # of samples, and you want to calculate the average time per cluster.